In [1]:
import os
import sys
import numpy as np
import random
import nltk
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout, Lambda
from keras.utils import np_utils, to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
from keras.models import load_model


Using TensorFlow backend.


In [2]:
# Load the text
# f = open(os.getcwd()+'/data/shakespeare.txt')
# text = f.readlines()

# text = open('data/shakespeare.txt', 'r').read()
# text = text.lower()

f = open(os.getcwd()+'/data/shakespeare.txt')
lines = f.readlines()
all_words = []
new_text = ""


# important grammatical markers include ,:;?. spaces and \n

for line in lines:
    line = line.strip()
    line = line.lower()
#     newPunct = "!#$%&()*'+,./:;<=>?@[\]^_`{|}~1234567890-' '"
    newPun = "#$%&()*+/<=>@[\]^_`{|}~1234567890-"
    if len(line) > 0:
        new_text += ''.join(c for c in line if c not in newPun) # changed from newPunct
    if line.isdigit()==False:
        all_words.append(line)

# text with structure
og_text = [x for x in all_words if x != []]

# correct characters and dictionary
chars = sorted(list(set(new_text)))
chars = [x for x in chars if x not in newPun] # changed from newPunct

# correct mappings
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = {i:ch for i, ch in enumerate(chars)}


n_chars = len(new_text)
n_vocab = len(chars)





In [3]:
num_chars = len(chars)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = new_text[i:i + seq_length]
	seq_out = new_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
# print "Total Patterns: ", n_patterns



In [4]:
# reshape X to be [samples, time steps, features]
X = np.array([to_categorical(seq, num_classes=n_vocab) for seq in dataX])
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


In [5]:
# 72915 x 40 
X.shape

(91310, 40, 34)

In [6]:
temp = 0.2

In [7]:
# I'm hard coding temp in the Lmbda line for ease of loading, temp = 0.2
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], num_chars)))
model.add(Dense(num_chars, activation='softmax'))
model.add(Lambda(lambda x: x / 0.2))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
model.fit(X, y, epochs=100, batch_size=128)

Epoch 1/100
91310/91310 [==============================] - 205s 2ms/step - loss: 2.9516
Epoch 2/100
91310/91310 [==============================] - 198s 2ms/step - loss: 2.9082
Epoch 3/100
91310/91310 [==============================] - 199s 2ms/step - loss: 2.6911
Epoch 4/100
91310/91310 [==============================] - 198s 2ms/step - loss: 2.5115
Epoch 5/100
91310/91310 [==============================] - 200s 2ms/step - loss: 2.3946
Epoch 6/100
91310/91310 [==============================] - 199s 2ms/step - loss: 2.3149
Epoch 7/100
91310/91310 [==============================] - 196s 2ms/step - loss: 2.2501
Epoch 8/100
91310/91310 [==============================] - 200s 2ms/step - loss: 2.1971
Epoch 9/100
91310/91310 [==============================] - 199s 2ms/step - loss: 2.1490
Epoch 10/100
91310/91310 [==============================] - 199s 2ms/step - loss: 2.1054
Epoch 11/100
91310/91310 [==============================] - 200s 2ms/step - loss: 2.0653
Epoch 12/100
91310/91310 [====

91310/91310 [==============================] - 198s 2ms/step - loss: 0.3349
Epoch 94/100
91310/91310 [==============================] - 198s 2ms/step - loss: 0.3261
Epoch 95/100
91310/91310 [==============================] - 197s 2ms/step - loss: 0.3203
Epoch 96/100
91310/91310 [==============================] - 199s 2ms/step - loss: 0.3221
Epoch 97/100
91310/91310 [==============================] - 197s 2ms/step - loss: 0.3054
Epoch 98/100
91310/91310 [==============================] - 198s 2ms/step - loss: 0.3065
Epoch 99/100
91310/91310 [==============================] - 197s 2ms/step - loss: 0.3084
Epoch 100/100
91310/91310 [==============================] - 197s 2ms/step - loss: 0.2909


In [9]:
model.save('my_rnn.h5')

# loaded_rnn = model_load('my_rnn.h5')

In [10]:
def generate_text(model, encoding, seed_text, num_chars, sequence_length, max_prob=False):
	'''
	This function uses the trained model and encoding map to generate text
	of a sonnet for the requested number of characters, each time using the last 
	sequence_length characters to predict the next character.
	Input:
		model: The trained model
		encoding: the mapping of characters to integers
		seed_text: the initial text used to predict the next characters
		num_chars: the number of characters we want to predict
		sequence_length: length of the sequences trained on
		max_prob: 
			True = take the highest probability character
			False = take a character based on the probability distribution
	
	Ouput:
		new_text: text with num_chars characters of generated text in 
				  addition to the seed text
	'''
	# Initialize with this seed text
	new_text = seed_text
	for _ in range(num_chars):
		# Convert current sequence from characters to integers
		sequence = [encoding[char] for char in new_text]
		# Only look at the newest 40 characters
		sequence = pad_sequences([sequence], maxlen=40, truncating='pre')
		# Apply one-hot encoding
		encoded_sequence = to_categorical(sequence, num_classes=len(encoding))
		if max_prob:
			# Predict character by max probability
			prob_index = model.predict_classes(encoded_sequence)
		else:
			# Predict character by probability distribution
			prob_indices = model.predict(encoded_sequence)[0]
			prob_index = np.random.choice(range(len(prob_indices)), p=prob_indices)
		# Convert integer back to character
		new_char = ''
		for char, index in encoding.items():
			if index == prob_index:
				new_char = char
				break
		new_text += new_char

	return new_text

In [11]:
def plotWordFrequency(input):
    f = open(artist_file,'r')
    words = [x for y in [l.split() for l in f.readlines()] for x in y]
    data = sorted([(w, words.count(w)) for w in set(words)], key = lambda x:x[1], reverse=True)[:40] 
    most_words = [x[0] for x in data]
    times_used = [int(x[1]) for x in data]
    plt.figure(figsize=(20,10))
    plt.bar(x=sorted(most_words), height=times_used, color = 'grey', edgecolor = 'black',  width=.5)
    plt.xticks(rotation=45, fontsize=18)
    plt.yticks(rotation=0, fontsize=18)
    plt.xlabel('Most Common Words:', fontsize=18)
    plt.ylabel('Number of Occurences:', fontsize=18)
    plt.title('Most Commonly Used Words: %s' % (artist_file), fontsize=24)
    plt.show()

In [12]:
# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_rnn.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_rnn_weights.h5")

In [13]:
temp = 0.2 # this is essential

# load json and create model
json_file = open('model_rnn.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_rnn_weights.h5")
print("Loaded model from disk")

# loaded_model.save('model_rnn.hdf5')
# loaded_model=load_model('model_rnn.hdf5') # this is complaining that 'temp' isn't initialized 

Loaded model from disk


/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [16]:
# will passing the model as 'model' work even if it can't load it?
sonnet = generate_text(model, char_to_int, seed_text='from creatures', num_chars=60, sequence_length=40, max_prob=True)

In [17]:
sonnet

'from creatures thour my fro be assth thour my gruth thour gruth thoull fro'